# Modelagem

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

import warnings

# remoção segura para deixar o log mais limpo
warnings.filterwarnings("ignore", message="X does not have valid feature names") 

## Pré-processamento dos dados para o modelo 

Leitura do dataset de features e covertendo os tipos dos dados

In [2]:
# Leitura do dataset
dataset_features_v1 = pd.read_csv('../data/processed/dataset_features_v1.csv')

# Conversão dos tipos numéricos e categóricos
dataset_features_v1 = dataset_features_v1.astype({
    'ID_CLIENTE': 'int64',
    'VALOR_A_PAGAR': 'float64',
    'TAXA': 'float64',
    'RENDA_MES_ANTERIOR': 'float64',
    'NO_FUNCIONARIOS': 'int64',
    'FLAG_PF': 'int64',
    'DIAS_ATRASO': 'int64',
    'TARGET_INADIMPLENCIA': 'int64',
    'DIAS_ADIANTAMENTO': 'int64',
    'TEMPO_DE_CASA_MESES': 'int64',
    'PRAZO_PAGAMENTO_DIAS': 'int64',
    'MES_SAFRA': 'int32',
    'INADIMPLENCIAS_ANTERIORES': 'int64',
    'SEGMENTO_INDUSTRIAL': 'category',
    'DOMINIO_EMAIL': 'category',
    'PORTE': 'category',
    'CEP_2_DIG': 'category',
})

# Conversão de datas
dataset_features_v1['SAFRA_REF'] = pd.to_datetime(
    dataset_features_v1['SAFRA_REF'], format='%Y-%m-%d', errors='coerce'
)

dataset_features_v1['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(
    dataset_features_v1['DATA_EMISSAO_DOCUMENTO'], format='%Y-%m-%d', errors='coerce'
)

dataset_features_v1['DATA_PAGAMENTO'] = pd.to_datetime(
    dataset_features_v1['DATA_PAGAMENTO'], format='%Y-%m-%d', errors='coerce'
)

dataset_features_v1['DATA_VENCIMENTO'] = pd.to_datetime(
    dataset_features_v1['DATA_VENCIMENTO'], format='%Y-%m-%d', errors='coerce'
)

dataset_features_v1['DATA_CADASTRO'] = pd.to_datetime(
    dataset_features_v1['DATA_CADASTRO'], format='%Y-%m-%d', errors='coerce'
)


In [3]:
dataset_features_v1.dtypes

ID_CLIENTE                            int64
SAFRA_REF                    datetime64[ns]
DATA_EMISSAO_DOCUMENTO       datetime64[ns]
DATA_PAGAMENTO               datetime64[ns]
DATA_VENCIMENTO              datetime64[ns]
VALOR_A_PAGAR                       float64
TAXA                                float64
RENDA_MES_ANTERIOR                  float64
NO_FUNCIONARIOS                       int64
DATA_CADASTRO                datetime64[ns]
FLAG_PF                               int64
SEGMENTO_INDUSTRIAL                category
DOMINIO_EMAIL                      category
PORTE                              category
CEP_2_DIG                          category
DIAS_ATRASO                           int64
TARGET_INADIMPLENCIA                  int64
DIAS_ADIANTAMENTO                     int64
TEMPO_DE_CASA_MESES                   int64
PRAZO_PAGAMENTO_DIAS                  int64
MES_SAFRA                             int32
INADIMPLENCIAS_ANTERIORES             int64
ADIANTAMENTOS_ANTERIORES        

#### Remoção de colunas irrelevantes

Comecei eliminando colunas que não agregam valor ao modelo.

* **ID_CLIENTE** é apenas um identificador único, não possui utilidade preditiva.
* As colunas de datas brutas (**DATA_EMISSAO_DOCUMENTO**, **DATA_PAGAMENTO**, **DATA_VENCIMENTO**, **DATA_CADASTRO** e **SAFRA_REF**) foram removidas porque já foram utilizadas para criar variáveis derivadas mais informativas.

#### Prevenção de data leakage

* As variáveis **DIAS_ATRASO** e **DIAS_ADIANTAMENTO** foram retiradas porque só estariam disponíveis após o vencimento, ou seja, depois da ocorrência ou não da inadimplência.
* Manter essas colunas geraria vazamento de informação (data leakage), prejudicando a generalização do modelo e provocando overfitting.

#### Codificação de variáveis categóricas

* As variáveis **SEGMENTO_INDUSTRIAL**, **PORTE** e **DOMINIO_EMAIL**, por serem categóricas com baixa cardinalidade, foram tratadas com One-Hot Encoding.
* Já a variável **CEP_2_DIG** apresenta alta cardinalidade, e por isso utilizei Frequency Encoding, uma técnica que substitui cada categoria pela frequência relativa com que aparece na base de dados. Isso evita a explosão de colunas sem perder a representatividade da variável.

#### Separação dos dados

* Após o pré-processamento, separei as variáveis explicativas (**X**) da variável alvo (**TARGET_INADIMPLENCIA**).
* Em seguida, fiz a divisão entre treino e teste com **train_test_split**, utilizando **stratify=y** para garantir a mesma proporção de inadimplentes em ambas as amostras.


In [4]:
# Eliminar colunas irrelevantes
cols_drop = [
    'ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO', 'DATA_PAGAMENTO',
    'DATA_VENCIMENTO', 'DATA_CADASTRO', 'SAFRA_REF',
    'DIAS_ATRASO', 'DIAS_ADIANTAMENTO', # Colunas que só são conhecidas depois do treinamento. São removidas para evitar data leakage
]

df = dataset_features_v1.drop(columns=cols_drop)


# Aplicando One-Hot Encoding em variáveis categóricas de baixa cardinalidade
df = pd.get_dummies(df, columns=['SEGMENTO_INDUSTRIAL', 'PORTE', 'DOMINIO_EMAIL'], drop_first=True)

# Aplicando Frequency Encoding para variável com alta cardinalidade 
freq_map = df['CEP_2_DIG'].value_counts(normalize=True)
df['CEP_2_DIG'] = df['CEP_2_DIG'].map(freq_map)

# Importante não adionar a variável target, resultaria em data leackage e consequentemente overfitting
X = df.drop(columns='TARGET_INADIMPLENCIA')
y = df['TARGET_INADIMPLENCIA']

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

## Escolhendo o modelo preditor e seus parâmetros

Explicação do que eu estou fazendo aqui embaixo

In [5]:
# Para o XGBoost.
# scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

'''
modelos_parametros = {
    "XGBoost": (
        XGBClassifier(
            #use_label_encoder=False,
            objective="binary:logistic",
            eval_metric="logloss",
            random_state=42,
            n_jobs=-1
        ),
        {
            "clf__n_estimators": [100, 200, 300],
            "clf__max_depth": [3, 5, 7, 10],
            "clf__learning_rate": [0.01, 0.05, 0.1],
            "clf__subsample": [0.6, 0.8, 1.0],
            "clf__colsample_bytree": [0.6, 0.8, 1.0],
            "clf__gamma": [0, 1, 5],  # controle da complexidade
            "clf__min_child_weight": [1, 5, 10],  # regularização de folhas
        }
    ),
    
    "RandomForest": (
        RandomForestClassifier(random_state=42, n_jobs=-1),
        {
            "clf__n_estimators": [100, 200, 300],
            "clf__max_depth": [None, 10, 20],
            "clf__min_samples_split": [2, 5, 10],
            "clf__min_samples_leaf": [1, 2, 4],
            "clf__max_features": ['sqrt', 'log2'],  # diversificação
            "clf__bootstrap": [True, False]
        }
    ),

    "LogisticRegression": (
        LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42),
        {
            "clf__C": [0.01, 0.1, 1.0, 10.0, 100.0]
        }
    ),

    "LightGBM": (
        LGBMClassifier(random_state=42),
        {
            "clf__n_estimators": [100, 200, 300],
            "clf__num_leaves": [31, 50, 100],
            "clf__learning_rate": [0.01, 0.05, 0.1],
            "clf__min_child_samples": [10, 20, 50],
            "clf__subsample": [0.6, 0.8, 1.0],
            "clf__colsample_bytree": [0.6, 0.8, 1.0]
        }
    )
}
'''


modelos_parametros = {
    "XGBoost": (
        XGBClassifier(
            objective="binary:logistic",
            eval_metric="logloss",
            random_state=42,
            n_jobs=-1,
            tree_method='gpu_hist',
            predictor='gpu_predictor',
            #use_label_encoder=False
        ),
        {
            "clf__n_estimators": [100, 200, 300, 400],
            "clf__max_depth": [3, 5, 7, 10, 12],
            "clf__learning_rate": [0.005, 0.01, 0.05, 0.1],
            "clf__subsample": [0.5, 0.6, 0.8, 1.0],
            "clf__colsample_bytree": [0.5, 0.6, 0.8, 1.0],
            "clf__gamma": [0, 1, 3, 5],  # controle da complexidade
            "clf__min_child_weight": [1, 3, 5, 10],
            "clf__reg_alpha": [0, 0.1, 0.5],  # L1 regularização
            "clf__reg_lambda": [1, 1.5, 2.0],  # L2 regularização
        }
    ),

    "RandomForest": (
        RandomForestClassifier(random_state=42, n_jobs=-1),
        {
            "clf__n_estimators": [100, 200, 300, 500],
            "clf__max_depth": [None, 10, 20, 30],
            "clf__min_samples_split": [2, 5, 10],
            "clf__min_samples_leaf": [1, 2, 4],
            "clf__max_features": ['sqrt', 'log2', None],
            "clf__bootstrap": [True, False],
            "clf__criterion": ['gini', 'entropy']
        }
    ),

    "LogisticRegression": (
        LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42),
        {
            "clf__C": [0.1, 1.0, 10.0]
        }
    ),
    "LightGBM": (
        LGBMClassifier(random_state=42, verbose=-1),
        {
            "clf__n_estimators": [100, 200],
            "clf__num_leaves": [31, 50],
            "clf__learning_rate": [0.01, 0.1]
        }
    )
}


# Validação cruzada e execução 
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
resultados = {}

for nome, (modelo, grid_params) in modelos_parametros.items():
    print(f"\nTreinando: {nome}")
    pipeline = Pipeline(steps=[
        ("scaler", StandardScaler()),
        ("smote", SMOTE(random_state=42)),
        ("clf", modelo)
    ])

    grid = GridSearchCV(
        pipeline,
        param_grid=grid_params,
        cv=cv,
        scoring="roc_auc",
        verbose=3,
        n_jobs=-1
    )

    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]

    resultados[nome] = {
        "Melhores parâmetros": grid.best_params_,
        "ROC AUC (validação)": grid.best_score_,
        "ROC AUC (teste)": roc_auc_score(y_test, y_proba),
        "Classification Report": classification_report(y_test, y_pred, output_dict=True),
        "Confusion Matrix": confusion_matrix(y_test, y_pred).tolist()
    }

    print(resultados)


Treinando: XGBoost
Fitting 5 folds for each of 184320 candidates, totalling 921600 fits


KeyboardInterrupt: 

In [ ]:
# Resultados Finais 
for nome, res in resultados.items():
    print(f"\n=== {nome} ===")
    print("Melhores parâmetros:", res["Melhores parâmetros"])
    print("ROC AUC (validação):", round(res["ROC AUC (validação)"], 4))
    print("ROC AUC (teste):", round(res["ROC AUC (teste)"], 4))
    print("Classification Report:")
    print(pd.DataFrame(res["Classification Report"]).T)
    print("Confusion Matrix:")
    print(np.array(res["Confusion Matrix"]))
    #plotando a matriz de confusão 
    #cm = np.array(res["Confusion Matrix"])
    #disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
    #disp.plot(cmap="plasma", values_format="d")
    #plt.title(f"Matriz de Confusão - {nome}")
    #plt.show()


=== XGBoost ===
Melhores parâmetros: {'clf__colsample_bytree': 0.6, 'clf__gamma': 0, 'clf__learning_rate': 0.1, 'clf__max_depth': 10, 'clf__min_child_weight': 1, 'clf__n_estimators': 300, 'clf__subsample': 0.6}
ROC AUC (validação): 0.9611
ROC AUC (teste): 0.9591
Classification Report:
              precision    recall  f1-score      support
0              0.974856  0.980987  0.977912  13675.00000
1              0.739479  0.680812  0.708934   1084.00000
accuracy       0.958940  0.958940  0.958940      0.95894
macro avg      0.857168  0.830900  0.843423  14759.00000
weighted avg   0.957569  0.958940  0.958157  14759.00000
Confusion Matrix:
[[13415   260]
 [  346   738]]

=== RandomForest ===
Melhores parâmetros: {'clf__bootstrap': False, 'clf__max_depth': 20, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 5, 'clf__n_estimators': 300}
ROC AUC (validação): 0.9618
ROC AUC (teste): 0.9618
Classification Report:
              precision    recall  f1-score 

In [ ]:
'''
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Gerar um dataset de exemplo
X, y = make_classification(n_samples=10000, n_features=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo com uso explícito de GPU
model = xgb.XGBClassifier(
    tree_method='gpu_hist',         # Ativa o uso da GPU
    predictor='gpu_predictor',      # Usa GPU também na predição
    use_label_encoder=False,
    eval_metric='logloss'
)

# Treinamento
model.fit(X_train, y_train)

# Predição
y_pred = model.predict(X_test)

# Confirmação
print("✅ Treinamento concluído com GPU.")
'''


c:\Users\Matheus\miniconda3\envs\py310\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


✅ Treinamento concluído com GPU.


In [ ]:
# Salva
with open("../data/processed/resultados_grid_search_cv_v5.pkl", "wb") as f:
    pickle.dump(resultados, f)

In [ ]:
'''
from imblearn.over_sampling import SMOTE

# Divide treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Aplica SMOTE apenas no treino
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

# Treina o modelo final
xgb = XGBClassifier(**best_params)
xgb.fit(X_resampled, y_resampled)

# Avalia no conjunto de teste real
y_pred = xgb.predict(X_test)
'''